### Introduction
In this section we will see how to create custom datasets  
like torchvision dataset gives us, from our own images
We will use Cats and Dogs Dataset for the same

In [5]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io

Start with creating a class that will help us load the data

In [6]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        #return length of the passed dataset
        return len(self.annotations)
    
    def __getitem__(self, index):
        #return an image along with its target
        img_path = os.path.join(self.root_dir,
                                self.annotations.iloc[index,0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,0]))
        
        if self.transform:
            image = self.transform(image)
        
        return (image, y_label)

### Creating and Init and Model

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using '+str(device)+' device')

using cuda device


In [13]:
#hyperparams
in_channel = 3
num_classes = 10
learning_rate = 1e-3
batch_size = 32
num_epochs = 1

In [17]:
#we will use googlenet inception model
model = torchvision.models.googlenet(pretrained=True)
model.to(device);

In [16]:
#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr = learning_rate)

### Loading the Dataset